In [1]:
%cd ..

/home/tox/GCNembedding


/home/tox/GCNembedding/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import re

In [2]:
from src.utils import dictEndpoint
from src.metrics import metrics as dictMetrics

In [3]:
from src.paths import DIR_RAWDATASET
datasets = {endpoint: pd.read_csv(os.path.join(DIR_RAWDATASET,f'endpoint={endpoint}.csv'))
        for endpoint in dictEndpoint
}
Y = {k:v["Label"] for k,v in datasets.items() }
# smis = {k:v["SMILES"] for k,v in datasets.items() }

In [4]:
from src.utils import thr_storage_path
fp = 'daylight'
thrs = pd.read_csv(thr_storage_path, index_col=0)[fp].squeeze()

In [5]:
from src.paths import DIR_INTRAIDXS
idxDir = "intraDomainIdxs"
idxs = {endpoint : np.loadtxt(os.path.join(DIR_INTRAIDXS,f'endpoint={endpoint}-fp={fp}-thr={thrs[endpoint]}.csv'), dtype=int)
             for endpoint in dictEndpoint }

In [6]:
Yfilt = {k:v[ idxs[k] ] for k,v in Y.items()}
# smisfilt = {k:v[ idxs[k] ] for k,v in smis.items()}

In [7]:
Yfilt["estro"].value_counts()

Label
0    1210
1      82
Name: count, dtype: int64

In [8]:
args_compiler=re.compile(r'([^-/]+)=([^-/]+)')
def args_dict_from(path):
    path,ext = os.path.splitext(path)
    args_dict=dict(args_compiler.findall(path))
    return args_dict

In [9]:
def calcResultsCV(path):
    args_dict = args_dict_from(path)
    labels = Yfilt[args_dict["endpoint"]].values
    scoresDF = pd.read_csv(path)
    df = pd.DataFrame()
    
    for _,cv in scoresDF.items():
        maskScores = cv.notna()
        
        # -1 are scores referred to non decision, so they are not converted in binary label
        maskBinary = cv>=0
        
        yBinary = labels[maskBinary]
        yScores = labels[maskScores]

        preds = cv.copy()[maskBinary].apply(lambda x: 1 if x>=.5 else 0)
        # -1 scores are converted in undecision score = 0.5
        scores = cv.copy()[maskScores].apply(lambda x: 0.5 if x==-1 else x)
        
        results = {metric:fnc(yBinary, preds, scores) for metric,fnc in dictMetrics.items() if metric!="ROCcurve"}
        results ["ROCcurve"] = dictMetrics["ROCcurve"](yScores, preds, scores)
        
        df = pd.concat([df,
                        pd.DataFrame([results])
                       ], ignore_index=True)

    df = df.assign(**args_dict)
    return df

In [10]:
from glob import glob

In [19]:
template_scores_path = 'experiments/model={model}/*/n_split=*/scores/*'

In [12]:
from src.configs import cv_configs, nn_configs

# KNN CV

In [22]:
glob(template_scores_path.format(model='KNN'))

[]

In [23]:
resultsKNN = pd.concat([
    calcResultsCV(path)
    for path in glob('experiments/model=KNN/*/n_split=*/scores.csv')
])

In [24]:
resultsKNN=resultsKNN.groupby(['endpoint']+list(cv_configs.keys())).describe()

In [25]:
resultsKNN.loc[:,(slice(None), ["mean","std"])]

Accuracy           Sensitivity            \
                                     mean       std        mean       std   
endpoint n_split test_size seed                                             
andro    30      0.1       0     0.668106  0.095694    0.426787  0.171210   
carcino  30      0.1       0     0.691232  0.111479    0.626891  0.148038   
chrom    30      0.1       0     0.605013  0.173192    0.559257  0.248436   
devtox   30      0.1       0     0.678812  0.176830    0.903815  0.071734   
estro    30      0.1       0     0.638353  0.157342    0.313532  0.309265   
hepa     30      0.1       0     0.577496  0.157019    0.590468  0.214985   
muta     30      0.1       0     0.688225  0.163224    0.757628  0.131852   
skin     30      0.1       0     0.557771  0.240340    0.403492  0.375526   

                                Specificity                  f1            \
                                       mean       std      mean       std   
endpoint n_split test_size seed                                             
andro    30      0.1       0       0.909426  0.047451  0.429363  0.166751   
carcino  30      0.1       0       0.755574  0.100349  0.616492  0.140230   
chrom    30      0.1       0       0.650770  0.168301  0.556544  0.241495   
devtox   30      0.1       0       0.453810  0.340045  0.891169  0.066155   
estro    30      0.1       0       0.963173  0.020610  0.319940  0.297734   
hepa     30      0.1       0       0.564524  0.168662  0.577313  0.188618   
muta     30      0.1       0       0.618822  0.237841  0.745873  0.132354   
skin     30      0.1       0       0.712050  0.218576  0.409356  0.374039   

                                 ROCcurve            
                                     mean       std  
endpoint n_split test_size seed                      
andro    30      0.1       0     0.645537  0.083585  
carcino  30      0.1       0     0.653608  0.077880  
chrom    30      0.1       0     0.581481  0.102804  
devtox   30      0.1       0     0.624852  0.143225  
estro    30      0.1       0     0.623600  0.135821  
hepa     30      0.1       0     0.560806  0.097169  
muta     30      0.1       0     0.629373  0.083978  
skin     30      0.1       0     0.561068  0.137057

# GCNN CV

In [27]:
resultsGCN = pd.concat([
    calcResultsCV(path)
    for path in glob('experiments/model=GCN*/*/n_split=*/scores/*')
])

In [28]:
resultsGCN=resultsGCN.groupby(['endpoint']+list(cv_configs|nn_configs.keys())).describe()

In [29]:
resultsGCN.loc[:,(slice(None), ["mean","std"])]

Accuracy  \
                                                                        mean   
endpoint hidden n_split embedding_size lr    test_size epochs seed             
andro    10     30      2              0.001 0.1       1000   0     0.738583   
carcino  10     30      2              0.001 0.1       1000   0     0.775758   
chrom    10     30      2              0.001 0.1       1000   0     0.632779   
devtox   10     30      2              0.001 0.1       1000   0     0.696755   
estro    10     30      2              0.001 0.1       1000   0     0.793475   
hepa     10     30      2              0.001 0.1       1000   0     0.612861   
muta     10     30      2              0.001 0.1       1000   0     0.702071   
skin     10     30      2              0.001 0.1       1000   0     0.607566   

                                                                              \
                                                                         std   
endpoint hidden n_split embedding_size lr    test_size epochs seed             
andro    10     30      2              0.001 0.1       1000   0     0.057620   
carcino  10     30      2              0.001 0.1       1000   0     0.062492   
chrom    10     30      2              0.001 0.1       1000   0     0.074516   
devtox   10     30      2              0.001 0.1       1000   0     0.097798   
estro    10     30      2              0.001 0.1       1000   0     0.099986   
hepa     10     30      2              0.001 0.1       1000   0     0.070365   
muta     10     30      2              0.001 0.1       1000   0     0.042747   
skin     10     30      2              0.001 0.1       1000   0     0.099049   

                                                                   Sensitivity  \
                                                                          mean   
endpoint hidden n_split embedding_size lr    test_size epochs seed               
andro    10     30      2              0.001 0.1       1000   0       0.531196   
carcino  10     30      2              0.001 0.1       1000   0       0.721496   
chrom    10     30      2              0.001 0.1       1000   0       0.601568   
devtox   10     30      2              0.001 0.1       1000   0       0.932949   
estro    10     30      2              0.001 0.1       1000   0       0.615729   
hepa     10     30      2              0.001 0.1       1000   0       0.619587   
muta     10     30      2              0.001 0.1       1000   0       0.702201   
skin     10     30      2              0.001 0.1       1000   0       0.473161   

                                                                              \
                                                                         std   
endpoint hidden n_split embedding_size lr    test_size epochs seed             
andro    10     30      2              0.001 0.1       1000   0     0.116440   
carcino  10     30      2              0.001 0.1       1000   0     0.102449   
chrom    10     30      2              0.001 0.1       1000   0     0.104807   
devtox   10     30      2              0.001 0.1       1000   0     0.031691   
estro    10     30      2              0.001 0.1       1000   0     0.197393   
hepa     10     30      2              0.001 0.1       1000   0     0.112117   
muta     10     30      2              0.001 0.1       1000   0     0.071167   
skin     10     30      2              0.001 0.1       1000   0     0.170352   

                                                                   Specificity  \
                                                                          mean   
endpoint hidden n_split embedding_size lr    test_size epochs seed               
andro    10     30      2              0.001 0.1       1000   0       0.945971   
carcino  10     30      2              0.001 0.1       1000   0       0.830019   
chrom    10     30      2              0.001 0.1       1000   0       0.663989   
devtox   10     30      2     